# 01. FEATURE EXTRACTION
---

# README

## directories
* All files cretaed or used in my noteboks require speciffic directory organization  
* it is as follow  
>    * basedir   
        * __models__ : contains downloaded tf hub model for feature extraction
        * __notebooks__ : jupiter noteboks 1-9    
        * __src__
            * __utils__ : contain .py files require for the analysis, loaded in each notebook, 
        * __data__
            * __raw__
                * __swissroads__      : data provided by EPFL-ext (created automatically)
            * __interim__ : intermediate and precessed data required for other tasks
                * __swissroads__       : extracted features, labels and logfiles (created automatically)
            * __results__ :  contains one or more directory for each jopiter notebook : resulst, eg: model predictions, 
                * __swissroads_cnn__
                * __swissroads_knn__
                * __swissroads_decision_trees__
                * etc...
                
## basedir:
- is is sufficinet to set up basedir in config section in eac h jupyter notebook torepeat the analsis   
- in case something is missing please contact __$prosikiewicz@gmail.com$__  

## config
- to make the control of each task simpler, I added config file to first cell of each notebook, 
- there are two parts:  
    - project config: used by all tasks 1-9  
    - task config: speciffic to a given task  


# Setup the enviroment

In [2]:
"""
    TO START, REPLACE basedir, with path to preffered location in your system
    THEN RUN THE COMMAND BELOW, and finally, copy Notebooks to notebook folder, 
    py file to src/utils, and tfhub to models folder. 
"""
# ----------------------------------------------------------------------------------------------------
import os
basedir = '/Users/pawel/Desktop/Activities/005__COURSES/000__EPFLext_ADSML/Module 4 __ Project/test'
# ----------------------------------------------------------------------------------------------------


files_to_create = {
    "for jupyter notebooks": os.path.join(basedir, "notebooks"),
    "for tfhub model": os.path.join(basedir, "models"),
    "for config and my tools": os.path.join(basedir, "src"),
    "IMPORTANT : HERE YOU MUST COPY ALL PY FILES IN UTILS": os.path.join(basedir, "src/utils"),
    # ....
    "to store data and resuls": os.path.join(basedir, "data"),
    "here copy swissroad files with raw images": os.path.join(basedir, "data/raw"),
    "to store matrices with extracted features": os.path.join(basedir, "data/interim"),
    "for final results": os.path.join(basedir, "data/results")
}
# ....
for file_function in  list(files_to_create.keys()):
    try:
        os.mkdir(files_to_create[file_function])
    except:
        print("file", file_function, "was already created")




file for jupyter notebooks was already created
file for tfhub model was already created
file for config and my tools was already created
file IMPORTANT : HERE YOU MUST COPY ALL PY FILES IN UTILS was already created
file to store data and resuls was already created
file here copy swissroad files with raw images was already created
file to store matrices with extracted features was already created
file for final results was already created


# Config

In [1]:
# ************************************************************************* #
#     Author:   Pawel Rosikiewicz                                           #       
#     Copyrith: IT IS NOT ALLOWED TO COPY OR TO DISTRIBUTE                  #
#               these file without written                                  #
#               persmission of the Author                                   #
#     Contact:  prosikiewicz@gmail.com                                      #
#                                                                           #
# ************************************************************************* #

# Config

# basedir
basedir = '/Users/pawel/Desktop/Activities/005__COURSES/000__EPFLext_ADSML/Module 4 __ Project/solution'

# project config
project_name = "Pawel_Rosikiewicz_EPFLext_Project4"
dataset_name = "swissroads"
subset_names = ["train", "valid", "test"]
module_names = ["imagenet"]
class_colors = {          "car": "orange",
                          "van": "steelblue",
                          "truck": "red",
                          "bike": "forestgreen",
                          "motorcycle" : "purple",
                           "other": "dimgrey"}

# task config
task_name = "feature_extraction"
subset_names_to_encode = ["train", "valid", "test"]
use_url = "no" # if yes, module_paths must contain url to tfhub modules, 
module_paths = ["imagenet_inception_v3_feature_vector_3"] # tfmodule names stored in basedir/models
img_imput_size = [(299, 299)] # required by imagenet, 
generator_batch_size = 10000 # nr of images that will be loaded in order of appearance to one batch with enc oded feastures, if less found, all will be used,

# Load packages

In [2]:
import os # allow changing, and navigating files and folders, 
import sys
import re # module to use regular expressions, 
import glob # lists names in folders that match Unix shell patterns
import random # functions that use and generate random numbers

import pickle
import warnings
import numpy as np # support for multi-dimensional arrays and matrices
import pandas as pd # library for data manipulation and analysis
import seaborn as sns # advance plots, for statistics, 
import matplotlib as mpl # to get some basif functions, heping with plot mnaking 
import matplotlib.pyplot as plt # for making plots, 

from PIL import Image, ImageDraw
from IPython.display import display
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load custom functions created for the project from basedir/src/utils

In [3]:
basedir = os.path.dirname(os.getcwd())
os.chdir(basedir)
sys.path.append(basedir)

# my tools, loaded only form basedir, 
from src.utils.feature_extraction_tools import encode_images
from src.utils.data_loaders import load_encoded_imgbatch_using_logfile, load_raw_img_batch
from src.utils.example_plots_after_clustering import plot_img_examples, create_spaces_between_img_clusters, plot_img_examples_from_dendrogram
from src.utils.annotated_pie_charts import annotated_pie_chart_with_class_and_group, prepare_img_classname_and_groupname
from src.utils.tools_for_plots import create_class_colors_dict
from src.utils.logreg_tools import my_logredCV, plot_examples_with_predictions_and_proba, plot_examples_with_predictions_and_proba_gamma
from src.utils.random_forest import random_forest_grid_search, plot_random_tree_summary_andor_table
from src.utils.model_summary_plots import plot_grid_acc_and_return_summary_df, visual_model_summary
from src.utils.SVM_tools import SVM_grid_search, plot_grid_acc_and_return_summary_df
from src.utils.tools_for_plots import create_class_colors_dict
from src.utils.model_summary_plots import plot_grid_acc_and_return_summary_df, visual_model_summary, model_gridsearch_summary_plots
from src.utils.knn_tools import knn_grid_search
from src.utils.image_augmentation import *

# one of my functions was generating warnign, on copies, despite using proper syntax, 
# warnings.filterwarnings("ignore")

# setup paths for the task

In [4]:
PATH_raw = os.path.join(basedir, "data/raw", dataset_name)
PATH_encoded = os.path.join(basedir, "data/interim", dataset_name)
PATH_results = os.path.join(basedir, "data/results", f"{dataset_name}_{task_name}") # not used in that task, but created in case I swoudl like to save metadata in the future, 
PATH_models = os.path.join(basedir, "models")

# create new direcory for task results, 
try: 
    os.mkdir(PATH_results)
    try:
        os.chdir(PATH_results)
        print("PATH_results was created")
    except:
        print("ERROR: PATH_results was not created correctly")
except: 
    print("file already exist, or PATH_results was not created correctly")

file already exist, or PATH_results was not created correctly


# Extract Features form images using tfhub

In [6]:
# encode,  
for one_module_name, one_module_path, one_img_input_size in zip(module_names, module_paths, img_imput_size):
    '''
        all data subsets found in load_dir will be encoded automatically, 
        - logfile will be created for a given datasets
        - batch_labels csv file and npz file with encoded features will be created for 
        each data subset will have:
        - 
    '''
    
    print(f"Ecoding:,{one_module_name}, {one_module_path}, {one_img_input_size}")
    
    # I am using modules saved in computer memory, thus I need to build fiull path to them, 
    if use_url=="no":
        one_module_full_path = os.path.join(PATH_models, one_module_path)
    else:
        one_module_full_path = one_module_path
    
    # extract features    
    encode_images(
        
        # .. dastaset name & directories, 
        dataset_name=dataset_name,# dataset name used when saving encoded files, logfiles and other things, related to encoding, 
        subset_names=subset_names_to_encode,# list, ust names of files in the load_dir, if any, 
        load_dir=PATH_raw,   # full path to input data, ie. file folder with either folders with images names after class names, or folders with subsetnames, and folders names after each class in them, 
        save_dir=PATH_encoded, # all new files, will be saved as one batch, with logfile, if None, load_dir will be used, 

        # .. encoding module parameters, 
        module_name=one_module_name, 
        module_location=one_module_full_path, # full path to a given module, or url, 
        img_target_size=one_img_input_size, # image resolution in pixels, 
        generator_batch_size =generator_batch_size, 
        generator_shuffle    =False, 

        # .. other, 
        save_files=True,
        verbose=True                             
    )

Ecoding:,imagenet, imagenet_inception_v3_feature_vector_3, (299, 299)
Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


................................................................................
 Creating DataGenerators for: swissroads; imagenet;
................................................................................

INFO:tensorflow:Saver not created because there are no variables in the graph to restore

................................................................................
 TF Graph;
Feature extraction Module, from: /Users/pawel/Desktop/Activities/005__COURSES/000__EPFLext_ADSML/Module 4 __ Project/solution/models/imagenet_inception_v3_feature_vector_3
................................................................................
 Ecoding imgages in one batch for < train > dataset;
Feature number = (280, 2048)
label table shape = (280, 4)
Saved as:
imagenet_swissroads_train_encoded.n